In [4]:
import scipy.io
import numpy as np
import pandas as pd
import os

In [8]:
file_path = r'D:\Internship\PPG\filteredandpeakdetectedsignals\S1denoised'
data = scipy.io.loadmat(file_path)
print(data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Mar 13 16:22:20 2023', '__version__': '1.0', '__globals__': [], 'd_noise_removed': array([[-0.11843242, -0.12225881, -0.1269756 , ..., -0.08550322,
        -0.10230441, -0.06230696]])}


In [9]:
data = data['d_noise_removed']
data.shape

(1, 1509)

In [11]:
file_path_1 = r'D:\Internship\PPG\filteredandpeakdetectedsignals\S7denoised'
data_1 = scipy.io.loadmat(file_path_1)['d_noise_removed']
print(data_1.shape)

(1, 7312)


In [12]:
file_path_2 = r'D:\Internship\PPG\filteredandpeakdetectedsignals\S6denoised'
data_6 = scipy.io.loadmat(file_path_2)['d_noise_removed']
print(data_6.shape)

(1, 7237)


In [23]:
squeezed_data = np.squeeze(data)
squeezed_data.shape

(1509,)

In [24]:
squeezed_data

array([-0.11843242, -0.12225881, -0.1269756 , ..., -0.08550322,
       -0.10230441, -0.06230696])

In [25]:
splitted_data = np.array_split(squeezed_data,3)

In [28]:
splitted_data[0].shape

(503,)

In [31]:
np.save("test_array.npy",splitted_data[0])

In [32]:
loaded_data = np.load("test_array.npy")
loaded_data.shape

(503,)

## PRE-PROCESSING LABELS

In [36]:
df = pd.read_csv(r'D:\Internship\PPG\raw_label.csv')
df.head()

,Session,Index,Timestamp,Date,Time,O2 Saturation,Beats / min,Perfusion Index,Pleth Variability,Breaths / min,Time Stamp,Unnamed: 11
0,1,1,1678092555,3/6/2023,11:49:15,96,69,5.8,--,--,6/3/2023 8:49,3/6/2023 8:49
1,1,2,1678092556,3/6/2023,11:49:16,96,68,6.0,--,--,6/3/2023 8:49,NaN
2,1,3,1678092557,3/6/2023,11:49:17,96,68,6.0,--,--,6/3/2023 8:49,NaN
3,1,4,1678092558,3/6/2023,11:49:18,97,68,6.1,--,--,6/3/2023 8:49,NaN
4,1,5,1678092559,3/6/2023,11:49:19,97,68,6.1,--,--,6/3/2023 8:49,NaN


In [41]:
df['Beats / min'].head()

0    69
1    68
2    68
3    68
4    68
Name: Beats / min, dtype: int64

In [44]:
# Initialize an empty list to store the mean values
means_list = []
# Calculate the number of chunks
num_chunks = len(df) // 10 + (1 if len(df) % 10 else 0)

for i in range(num_chunks):
    # Calculate start and end indices for each chunk of 10 rows
    start_idx = i * 10
    end_idx = start_idx + 10
    
    # Calculate the mean of the current chunk for "Beats / min" column
    chunk_mean = df['Beats / min'][start_idx:end_idx].mean()
    
    # Append the mean to the list, if the chunk is not empty
    if not np.isnan(chunk_mean):
        means_list.append(chunk_mean)

# Create a new DataFrame for the mean values
mean_df = pd.DataFrame(means_list, columns=['labels'])

# Save the resulting DataFrame to a new CSV file
mean_df.to_csv(r'D:\Internship\PPG\labels.csv', index=False)

print(f"File saved successfully with {len(mean_df)} records.")


File saved successfully with 570 records.


In [38]:
# Ensure that the "Beats / min" column is of a numeric type
# df['Beats / min'] = pd.to_numeric(df['Beats / min'], errors='coerce')

# Calculate the mean of every 10 values in the 'Beats / min' column
means = df['Beats / min'].groupby(np.floor(np.arange(len(df)) / 10)).mean().reset_index(drop=True)

# Create a new DataFrame for storing the mean values with the column name 'labels'
mean_df = pd.DataFrame(means, columns=['labels'])

# Save the resulting DataFrame to a new CSV file
mean_df.to_csv(r'D:\Internship\PPG\labels.csv', index=False)

# PRE-PROCESSING FEATURES

In [4]:
import scipy.io
import numpy as np
import os

In [7]:
input_dir = r'D:\Internship\PPG\filteredandpeakdetectedsignals'
output_dir = r'D:\Internship\PPG\processed_2'

In [8]:
files = os.listdir(input_dir)
print(files)

['S10denoised.mat', 'S11denoised.mat', 'S12denoised.mat', 'S13denoised.mat', 'S14denoised.mat', 'S15denoised.mat', 'S16denoised.mat', 'S17denoised.mat', 'S18denoised.mat', 'S19denoised.mat', 'S1denoised.mat', 'S20denoised.mat', 'S21denoised.mat', 'S2denoised.mat', 'S3denoised.mat', 'S4denoised.mat', 'S5denoised.mat', 'S6denoised.mat', 'S7denoised.mat', 'S8denoised.mat', 'S9denoised.mat', 'vPPG_sync.asv', 'vPPG_sync.m']


In [9]:
for file in files:
    if file.endswith('.mat'):
        file_path = os.path.join(input_dir,file)
        data = scipy.io.loadmat(file_path)['d_noise_removed']
        data = data.squeeze() #remove 1st dimension
        
        splits = int(len(data)/250)
        sub_data = np.array_split(data,splits)
        
        for index,sub_array in enumerate(sub_data):
            np.save(f"{output_dir}\{file}_{index}",sub_array)

## REGRESSION MODEL

In [4]:
# Specify the directory containing your .npy files
npy_files_dir = r'D:\Internship\PPG\Vitals_Using_PPG\dataset\features'

# List all files in the directory and filter for .npy files
npy_files = [file for file in os.listdir(npy_files_dir) if file.endswith('.npy')]

# Sort the files if they are in a sequential order but named differently
# npy_files.sort()

# Load the .npy files
features = []
for file in npy_files:
    filepath = os.path.join(npy_files_dir, file)
    sample = np.load(filepath)
    
    # Truncate or keep the sample to the first 250 elements
    if sample.shape[0] > 250:
        truncated_sample = sample[:250]
    else:
        truncated_sample = sample
    
    features.append(truncated_sample)

# Convert the list of numpy arrays into a single numpy array
X = np.array(features)

# Load labels
labels_path = r'D:\Internship\PPG\Vitals_Using_PPG\dataset/labels.csv'
y = pd.read_csv(labels_path)

# Assuming your labels are in the first column
y = y['labels'].values


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

LinearRegression()

In [8]:
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.metrics import mean_absolute_error

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))
# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')

Mean squared error: 10937389.66
Coefficient of determination: -111422.31
Mean Absolute Error: 483.00


# Unfiltered Data

In [1]:
import scipy.io
import numpy as np
import pandas as pd
import os

In [2]:
file_path = r'D:\Internship\PPG\vppg kaust 2023 dataset rbg\s1_r'
data = scipy.io.loadmat(file_path)
print(data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Mar 13 21:26:28 2023', '__version__': '1.0', '__globals__': [], 'pulseSigr': array([[67.55822386, 67.63502845, 67.58110147, ..., 63.14719377,
        63.12000482, 63.16973717]])}


In [3]:
data = data['pulseSigr']
data.shape

(1, 1509)

##### Number of samples in filtered and unfiltered data is same

### Pre-processing 

In [8]:
input_dir = r'D:\Internship\PPG\vppg kaust 2023 dataset rbg'
output_dir = r'D:\Internship\PPG\vppg raw G channel processed'

files = [f for f in os.listdir(input_dir) if f.endswith('_g.mat')]

for file in files:
    if file.endswith('.mat'):
        file_path = os.path.join(input_dir,file)
        data = scipy.io.loadmat(file_path)['pulseSigg']
        data = data.squeeze() #remove 1st dimension
        
        splits = int(len(data)/250)
        sub_data = np.array_split(data,splits)
        
        for index,sub_array in enumerate(sub_data):
            np.save(f"{output_dir}\{file}_{index}",sub_array)

### Regression Model

In [10]:
# Specify the directory containing your .npy files
npy_files_dir = r'D:\Internship\PPG\vppg raw B channel processed'

# List all files in the directory and filter for .npy files
npy_files = [file for file in os.listdir(npy_files_dir) if file.endswith('.npy')]

# Sort the files if they are in a sequential order but named differently
# npy_files.sort()

# Load the .npy files
features = []
for file in npy_files:
    filepath = os.path.join(npy_files_dir, file)
    sample = np.load(filepath)
    
    # Truncate or keep the sample to the first 250 elements
    if sample.shape[0] > 250:
        truncated_sample = sample[:250]
    else:
        truncated_sample = sample
    
    features.append(truncated_sample)

# Convert the list of numpy arrays into a single numpy array
X = np.array(features)

# Load labels
labels_path = r'D:\Internship\PPG\Vitals_Using_PPG\dataset/labels.csv'
y = pd.read_csv(labels_path)

# Assuming your labels are in the first column
y = y['labels'].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.metrics import mean_absolute_error

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))
# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')

Mean squared error: 368.50
Coefficient of determination: -2.75
Mean Absolute Error: 13.70


# Filtering before Regression

In [1]:
import numpy as np
import os
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.metrics import mean_absolute_error

In [11]:
# Function to apply a low-pass Butterworth filter
def butter_lowpass(cutoff, nyquist, order=5):
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def lowpass_filter(data, cutoff, nyquist, order=5):
    b, a = butter_lowpass(cutoff, nyquist, order=order)
    y = lfilter(b, a, data)
    return y

npy_files_dir = r'D:\Internship\PPG\vppg raw b channel processed'
npy_files = [file for file in os.listdir(npy_files_dir) if file.endswith('.npy')]

features = []
for file in npy_files:
    filepath = os.path.join(npy_files_dir, file)
    sample = np.load(filepath)
    
    if sample.shape[0] > 250:
        truncated_sample = sample[:250]
    else:
        truncated_sample = sample
    
    cutoff_frequency =  0.5 # Cutoff frequency in Hz
    nyquist_frequency = 15  # Nyquist frequency in Hz, which is half the sampling rate
    filtered_sample = lowpass_filter(truncated_sample, cutoff_frequency, nyquist_frequency)
    
    features.append(filtered_sample)

# Convert the list of numpy arrays into a single numpy array
X = np.array(features)

# Load labels
labels_path = r'D:\Internship\PPG\Vitals_Using_PPG\dataset/labels.csv'
y = pd.read_csv(labels_path)

# Assuming your labels are in the first column
y = y['labels'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))
# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')


Mean squared error: 341.86
Coefficient of determination: -2.48
Mean Absolute Error: 12.64


# Other Regression Algos

### LGB

In [8]:
import lightgbm as lgb

In [10]:
lgb_model = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=100)
lgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l1')
y_pred_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration_)
print('LightGBM Regressor')
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred_lgb))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred_lgb))
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred_lgb):.2f}')


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38250
[LightGBM] [Info] Number of data points in the train set: 456, number of used features: 250
[LightGBM] [Info] Start training from score 72.781798
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

### SVR

In [2]:
from sklearn.svm import SVR

In [5]:
# Function to apply a low-pass Butterworth filter
def butter_lowpass(cutoff, nyquist, order=5):
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def lowpass_filter(data, cutoff, nyquist, order=5):
    b, a = butter_lowpass(cutoff, nyquist, order=order)
    y = lfilter(b, a, data)
    return y

npy_files_dir = r'D:\Internship\PPG\vppg raw b channel processed'
npy_files = [file for file in os.listdir(npy_files_dir) if file.endswith('.npy')]

features = []
for file in npy_files:
    filepath = os.path.join(npy_files_dir, file)
    sample = np.load(filepath)
    
    if sample.shape[0] > 250:
        truncated_sample = sample[:250]
    else:
        truncated_sample = sample
    
    cutoff_frequency =  0.5 # Cutoff frequency in Hz
    nyquist_frequency = 15  # Nyquist frequency in Hz, which is half the sampling rate
    filtered_sample = lowpass_filter(truncated_sample, cutoff_frequency, nyquist_frequency)
    
    features.append(filtered_sample)

# Convert the list of numpy arrays into a single numpy array
X = np.array(features)

# Load labels
labels_path = r'D:\Internship\PPG\Vitals_Using_PPG\dataset/labels.csv'
y = pd.read_csv(labels_path)

# Assuming your labels are in the first column
y = y['labels'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svr_model = SVR(C=1.0, epsilon=0.2)
svr_model.fit(X_train, y_train)
y_pred_svr = svr_model.predict(X_test)
print('Support Vector Regression')
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred_svr))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred_svr))
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred_svr):.2f}')


Support Vector Regression
Mean squared error: 98.49
Coefficient of determination: -0.00
Mean Absolute Error: 8.02


### Random Forest Regressor

In [6]:
from sklearn.ensemble import RandomForestRegressor

In [7]:
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print('Random Forest Regressor')
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred_rf))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred_rf))
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred_rf):.2f}')


Random Forest Regressor
Mean squared error: 74.93
Coefficient of determination: 0.24
Mean Absolute Error: 6.52
